In [2]:
from datetime import datetime
import pandas as pd
import requests
import json

url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
df_listing = pd.read_html(url, header=0)[0]
rename_cols = {'회사명':'Name', '종목코드':'Symbol', '업종':'Sector', '주요제품':'Industry', '상장일':'ListingDate',  \
       '결산월':'SettleMonth', '대표자명':'Representative', '홈페이지':'Homepage', '지역':'Region'}

df_listing = df_listing.rename(columns = rename_cols)

df_listing['Symbol'] = df_listing['Symbol'].apply(lambda x : '{:06d}'.format(x))
df_listing['ListingDate'] = pd.to_datetime(df_listing['ListingDate'])

data = {'bld': 'dbms/comm/finder/finder_stkisu',}
r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', data=data)

jo = json.loads(r.text)
df_finder = pd.json_normalize(jo, 'block1')

df_finder.columns = ['FullCode', 'Symbol', 'Name', 'MarketCode', 'MarketName', 'Market', 'Ord1', 'Ord2']

df_right = df_finder[['Symbol', 'Market', 'Name']]
df_left = df_listing[['Symbol', 'Sector', 'Industry', 'ListingDate', 'SettleMonth', 'Representative', 'Homepage', 'Region']]

df_master = pd.merge(df_left, df_right, how='left', left_on='Symbol', right_on='Symbol')

print(df_master[['Symbol', 'Name', 'Market','Sector']].head(100))


    Symbol       Name  Market          Sector
0   004840      DRB동일   KOSPI        고무제품 제조업
1   155660        DSR   KOSPI     1차 비철금속 제조업
2   078930         GS   KOSPI          기타 금융업
3   001250      GS글로벌   KOSPI       상품 종합 도매업
4   294870  HDC현대산업개발   KOSPI          건물 건설업
..     ...        ...     ...             ...
95  067290       JW신약  KOSDAQ         의약품 제조업
96  024120     KB오토시스  KOSDAQ   자동차 신품 부품 제조업
97  060720       KH바텍  KOSDAQ        전자부품 제조업
98  024940       PN풍년  KOSDAQ  기타 금속 가공제품 제조업
99  036120    SCI평가정보  KOSDAQ      기타 정보 서비스업

[100 rows x 4 columns]
